In [7]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, auc
import glob
import matplotlib.pyplot as plt
from time import strftime
from hep_ml import uboost
import pandas as pd
from matplotlib.colors import LogNorm

In [8]:
# Truth matched variables order 
# in signal events
'''
0.    subl.girth, 
1.    subl.ptD, 
2.    subl.axismajor, 
3.    subl.axisminor,
4.    subl.ecfM2b1, 
5.    subl.ecfD2b1, 
6.    subl.ecfC2b1, 
7.    subl.ecfN2b2,
8.    subl.metdphi,
9.    subl.pt, 
10.   subl.eta, 
11.   subl.phi, 
12.   subl.energy, 
13.   subl.rt, 
14.   subl.mt, 
15.   subl.multiplicity, 
16.   event[b'MET']
'''

"\n0.    subl.girth, \n1.    subl.ptD, \n2.    subl.axismajor, \n3.    subl.axisminor,\n4.    subl.ecfM2b1, \n5.    subl.ecfD2b1, \n6.    subl.ecfC2b1, \n7.    subl.ecfN2b2,\n8.    subl.metdphi,\n9.    subl.pt, \n10.   subl.eta, \n11.   subl.phi, \n12.   subl.energy, \n13.   subl.rt, \n14.   subl.mt, \n15.   subl.multiplicity, \n16.   event[b'MET']\n"

In [9]:
qcd_bin = [np.load('/home/snabili/Data_ControlRegion/svj-bdt/npzfiles/allbdt_var/qcd_bin'+str(n) +'.npz') for n in range(0,9)]
ttjets_bin = [np.load('/home/snabili/Data_ControlRegion/svj-bdt/npzfiles/allbdt_var/ttjet_bin'+str(n) +'.npz') for n in range(0,8)]


labels_extended_scenario4 = ['subl.girth', 'subl.ptD', 'subl.axismajor', 'subl.axisminor', 
        'subl.ecfM2b1', 'subl.ecfD2b1', 'subl.ecfC2b1', 
        'subl.ecfN2b2', 'subl.metdphi', 'eta', 'phi', 'mt']


QCD0_ext_sc4 = np.transpose(np.array([qcd_bin[0][k][qcd_bin[0]['rt']>1.1] for k in labels_extended_scenario4]))
QCD1_ext_sc4 = np.transpose(np.array([qcd_bin[1][k][qcd_bin[1]['rt']>1.1] for k in labels_extended_scenario4]))
QCD2_ext_sc4 = np.transpose(np.array([qcd_bin[2][k][qcd_bin[2]['rt']>1.1] for k in labels_extended_scenario4]))
QCD3_ext_sc4 = np.transpose(np.array([qcd_bin[3][k][qcd_bin[3]['rt']>1.1] for k in labels_extended_scenario4]))
QCD4_ext_sc4 = np.transpose(np.array([qcd_bin[4][k][qcd_bin[4]['rt']>1.1] for k in labels_extended_scenario4]))
QCD5_ext_sc4 = np.transpose(np.array([qcd_bin[5][k][qcd_bin[5]['rt']>1.1] for k in labels_extended_scenario4]))
QCD6_ext_sc4 = np.transpose(np.array([qcd_bin[6][k][qcd_bin[6]['rt']>1.1] for k in labels_extended_scenario4]))
QCD7_ext_sc4 = np.transpose(np.array([qcd_bin[7][k][qcd_bin[7]['rt']>1.1] for k in labels_extended_scenario4]))
QCD8_ext_sc4 = np.transpose(np.array([qcd_bin[8][k][qcd_bin[8]['rt']>1.1] for k in labels_extended_scenario4]))

TTJets0_ext_sc4 = np.transpose(np.array([ttjets_bin[0][k][ttjets_bin[0]['rt']>1.1] for k in labels_extended_scenario4]))
TTJets1_ext_sc4 = np.transpose(np.array([ttjets_bin[1][k][ttjets_bin[1]['rt']>1.1] for k in labels_extended_scenario4]))
TTJets2_ext_sc4 = np.transpose(np.array([ttjets_bin[2][k][ttjets_bin[2]['rt']>1.1] for k in labels_extended_scenario4]))
TTJets3_ext_sc4 = np.transpose(np.array([ttjets_bin[3][k][ttjets_bin[3]['rt']>1.1] for k in labels_extended_scenario4]))
TTJets4_ext_sc4 = np.transpose(np.array([ttjets_bin[4][k][ttjets_bin[4]['rt']>1.1] for k in labels_extended_scenario4]))
TTJets5_ext_sc4 = np.transpose(np.array([ttjets_bin[5][k][ttjets_bin[5]['rt']>1.1] for k in labels_extended_scenario4]))
TTJets6_ext_sc4 = np.transpose(np.array([ttjets_bin[6][k][ttjets_bin[6]['rt']>1.1] for k in labels_extended_scenario4]))
TTJets7_ext_sc4 = np.transpose(np.array([ttjets_bin[7][k][ttjets_bin[7]['rt']>1.1] for k in labels_extended_scenario4]))


In [10]:
def get_bkg_features_ext(sets_of_npzs, weights, n_target_events):
    weights /= np.sum(weights) # normalize
    n_events_per_set = (weights * n_target_events).astype(np.int32)
    X_combined = []
    for n_events, npzs in zip(n_events_per_set, sets_of_npzs):
        n_events_todo = n_events
        if npzs.shape[0] > n_events:
            X = npzs[:n_events]
        X_combined.append(X)
    X_final = np.vstack(X_combined)
    assert len(X_final.shape) == 2
    assert X_final.shape[1] == 12 # Scenario IV eta, phi and mt included
    print(X_final.shape)
    return X_final

In [11]:
def get_sig_features_ext(sets_of_npzs, weights, n_target_events):
    weights /= np.sum(weights) # normalize
    n_events_per_set = (weights * n_target_events).astype(np.int32)
    X_combined = []
    l1 = np.append(np.arange(0,9,1), np.arange(10,12,1)) # Scenario IV including eta, phi 
    l = np.append(l1,14) # appending mt 
    for n_events, npzs in zip(n_events_per_set, sets_of_npzs):
        n_events_todo = n_events
        for npz in npzs:
            if np.load(npz)['X'].shape[0] == 0: continue
            X = np.load(npz)['X'][:,l]
            n_events_this_npz = X.shape[0]
            if n_events_this_npz > n_events_todo:
                X = X[:n_events_todo]
            X_combined.append(X)
            n_events_todo -= X.shape[0]
            if n_events_todo == 0: break
        else:
            print(f'Warning: reached end of set of files with n_events_todo={n_events_todo}')

    X_final = np.vstack(X_combined)
    assert len(X_final.shape) == 2
    assert X_final.shape[1] == 12 # Scenario IV eta, phi and mt included
    print(X_final.shape)
    return X_final

In [12]:
labels_sig = ['mz250', 'mz350', 'mz450']
sig_npzfiles = [ glob.iglob(f'/mnt/hadoop/cms/store/user/snabili/UltraLegacy_npz/PreBDT_Signal_10312022_highstat_met_TREEMAKER/madpt300_{l}_mdark10_rinv0.3/*.npz') for l in labels_sig]
sig_xsec = [1, 1, 1]

qcd_xsec = np.array([7823, 648.2, 186.9, 32.293, 9.4183, 0.84265, .114943, .00682981, .000165445])
ttjets_xsec = np.array([831.76, 182.72, 182.72, 88.34, 2.685, 1.096, 0.194, 0.002])

bkg_xsec = np.append(qcd_xsec, ttjets_xsec)

qcd_eff    = np.array([len(qcd_bin[i]['pt'][qcd_bin[i]['rt']>1.1])/qcd_bin[i]['total'] for i in range(0, len(qcd_bin))])
ttjets_eff = np.array([len(ttjets_bin[i]['pt'][ttjets_bin[i]['rt']>1.1])/ttjets_bin[i]['total'] for i in range(0, len(ttjets_bin))])

bkg_eff = np.append(qcd_eff, ttjets_eff)
crosssections = bkg_eff * bkg_xsec

#Scenario II
sets_of_npzs_ext_sc4 = [QCD0_ext_sc4, QCD1_ext_sc4, QCD2_ext_sc4, QCD3_ext_sc4, QCD4_ext_sc4, QCD5_ext_sc4, QCD6_ext_sc4, QCD7_ext_sc4, QCD8_ext_sc4, TTJets0_ext_sc4, TTJets1_ext_sc4, TTJets2_ext_sc4, TTJets3_ext_sc4, TTJets4_ext_sc4, TTJets5_ext_sc4, TTJets6_ext_sc4, TTJets7_ext_sc4]
X_bkg_ext_sc4 = get_bkg_features_ext(sets_of_npzs_ext_sc4, crosssections, 50000)
y_bkg_ext_sc4 = np.zeros(X_bkg_ext_sc4.shape[0])
X_sig_ext_sc4 = get_sig_features_ext(sig_npzfiles, sig_xsec,10000)
y_sig_ext_sc4 = np.ones(X_sig_ext_sc4.shape[0])

(49992, 12)
(9999, 12)


In [13]:
X_ext_sc4 = np.vstack((X_sig_ext_sc4, X_bkg_ext_sc4))
y_ext_sc4 = np.concatenate((y_sig_ext_sc4, y_bkg_ext_sc4)).astype(np.int8)
print(X_ext_sc4.shape, y_ext_sc4.shape)
print(f'y: {y_ext_sc4}')
print(f'y.shape[0]/np.sum(y): {y_ext_sc4.shape[0]/np.sum(y_ext_sc4)}')
X_train_ext_sc4, X_test_ext_sc4, y_train_ext_sc4, y_test_ext_sc4 = train_test_split(X_ext_sc4, y_ext_sc4, test_size=.5)
y_train_ext_sc4 = y_train_ext_sc4.astype(np.int8)
y_test_ext_sc4 = y_test_ext_sc4.astype(np.int8)


(59991, 12) (59991,)
y: [1 1 1 ... 0 0 0]
y.shape[0]/np.sum(y): 5.999699969997


In [14]:
cls_sc4 = ['subl.girth', 'subl.ptD', 'subl.axismajor', 'subl.axisminor', 
        'subl.ecfM2b1', 'subl.ecfD2b1', 'subl.ecfC2b1', 
        'subl.ecfN2b2', 'subl.metdphi', 'eta', 'phi', 'mt']

pd_x_train_ext_sc4 = pd.DataFrame(X_train_ext_sc4, columns=cls_sc4)
pd_x_test_ext_sc4  = pd.DataFrame(X_test_ext_sc4,  columns=cls_sc4)

In [ ]:
base_tree = uboost.DecisionTreeClassifier(max_depth=4)
model_sc4 = uboost.uBoostClassifier(uniform_features=['mt'], uniform_label=0, base_estimator=base_tree, train_features=cls_sc4[:len(cls_sc4)], n_estimators=100)
model_sc4.fit(pd_x_train_ext_sc4, y_train_ext_sc4)


In [ ]:
y_pred_ext_sc4 = model_sc4.predict(pd_x_test_ext_sc4)
y_prob_ext_sc4 = model_sc4.predict_proba(pd_x_test_ext_sc4)[:,1]
y_prob_sig_ext_sc4 = y_prob_ext_sc4[y_test_ext_sc4 == 1]
y_prob_bkg_ext_sc4 = y_prob_ext_sc4[y_test_ext_sc4 == 0]

In [ ]:
fig, (ax1) = plt.subplots(1, 1, sharey=True, figsize=(10, 6))
fig.suptitle('Bkg(QCD+TTJets) & Sig(truth-matched) uBDT scores ($\eta_{J_2}, \phi_{J_2}$)', fontsize=18, y = 0.97)

ax1.hist(y_prob_sig_ext_sc4, label='sig', density=True, bins=15)
ax1.hist(y_prob_bkg_ext_sc4, label='qcd', alpha=.5, density=True, bins=15)
ax1.legend(fontsize=18)
ax1.grid()
ax1.set_xlabel('uBDT score', fontsize=18)
ax1.set_ylabel('A.U.', fontsize=18)
plt.savefig('ubdt_vs_bdt_study/ubdt_scores_scenario4.png')


In [ ]:
fig, (ax1) = plt.subplots(1, 1, sharey=True, figsize=(10, 6))
fig.suptitle('Bkg(QCD+TTJets) $M_{T,J_2}$ with uBDT cuts', fontsize=18, y = 0.99)

e_sc4 = X_test_ext_sc4.shape[1]-1
ax1.hist(X_test_ext_sc4[:,e_sc4][(y_test_ext_sc4==0) & (y_prob_ext_sc4>0.7)], bins=30, label='ubdt>0.70', density=True)
ax1.hist(X_test_ext_sc4[:,e_sc4][(y_test_ext_sc4==0) & (y_prob_ext_sc4>0.65)], bins=30, label='ubdt>0.65', density=True, alpha=0.4)
ax1.hist(X_test_ext_sc4[:,e_sc4][ y_test_ext_sc4==0], bins=30, alpha=0.4, label='no ubdt', density=True)
ax1.legend(fontsize=18)
ax1.set_title('$\eta_{J_2}, \phi_{J_2}$')
ax1.grid()
ax1.set_ylabel('A.U.', fontsize=18)
ax1.set_xlabel('$M_{T,J_2}$ (GeV)', fontsize=18)
plt.savefig('ubdt_vs_bdt_study/MT_uBDT_scenario4.png')


In [ ]:
fig = plt.figure(figsize=(10,6))
e_sc4 = X_test_ext_sc4.shape[1]-1
plt.hist2d(X_test_ext_sc4[:,e_sc4][y_test_ext_sc4==0],y_prob_ext_sc4[y_test_ext_sc4==0], bins=40, norm = LogNorm())
plt.colorbar()
plt.xlabel('$M_{T,J2}$ (GeV)', fontsize=18)
plt.ylabel('uBDT', fontsize=18)
plt.grid()
plt.title('Bkg(QCD+TTJets) uBDT vs $M_{T,J_2}$ ($\eta_{J_2}, \phi_{J_2}$)', fontsize=18)
plt.savefig('ubdt_vs_bdt_study/ubdt_vs_mt_scenario4.png')

In [ ]:
e_sc4 = X_test_ext_sc4.shape[1]-1
b = np.arange(0,1500,50)
ubdt0p70_sc4 = plt.hist(X_test_ext_sc4[:,e_sc4][(y_test_ext_sc4==0) & (y_prob_ext_sc4>0.70)], bins=b, label='ubdt>0.70', density=False)
ubdt0p65_sc4 = plt.hist(X_test_ext_sc4[:,e_sc4][(y_test_ext_sc4==0) & (y_prob_ext_sc4>0.65)], bins=b, label='ubdt>0.65', density=False, alpha=0.4)
ubdt0p60_sc4 = plt.hist(X_test_ext_sc4[:,e_sc4][(y_test_ext_sc4==0) & (y_prob_ext_sc4>0.60)], bins=b, label='ubdt>0.60', density=False, alpha=0.4)
ubdt_sc4 = plt.hist(X_test_ext_sc4[:,e_sc4][(y_test_ext_sc4==0)], bins=b, label='no ubdt', density=False, alpha=0.4)

plt.close()

ratio_0p60_sc4 = np.divide(ubdt0p60_sc4[0], ubdt_sc4[0], where=(ubdt_sc4[0]>0))
ratio_0p65_sc4 = np.divide(ubdt0p65_sc4[0], ubdt_sc4[0], where=(ubdt_sc4[0]>0))
ratio_0p70_sc4 = np.divide(ubdt0p70_sc4[0], ubdt_sc4[0], where=(ubdt_sc4[0]>0))

rotio_0p60_sc4 = ratio_0p60_sc4[(ubdt0p60_sc4[0]>0) & (ratio_0p60_sc4 > 0)]
rotio_0p65_sc4 = ratio_0p65_sc4[(ubdt0p65_sc4[0]>0) & (ratio_0p65_sc4 > 0)]
rotio_0p70_sc4 = ratio_0p70_sc4[(ubdt0p70_sc4[0]>0) & (ratio_0p70_sc4 > 0)]

edges_0p60_sc4 = ubdt_sc4[1][:len(ubdt_sc4[1])-1][(ubdt_sc4[0]>0) & (ratio_0p60_sc4 > 0)]
edges_0p65_sc4 = ubdt_sc4[1][:len(ubdt_sc4[1])-1][(ubdt_sc4[0]>0) & (ratio_0p65_sc4 > 0)]
edges_0p70_sc4 = ubdt_sc4[1][:len(ubdt_sc4[1])-1][(ubdt_sc4[0]>0) & (ratio_0p70_sc4 > 0)]



fig, (ax1) = plt.subplots(1, 1, sharey=True, figsize=(10, 6))
fig.suptitle('Bkg eff uBDT cuts ($\eta_{J_2}, \phi_{J_2}, p_{T,J_2}$)', fontsize=18, y = 1)

ax1.plot(edges_0p60_sc4, rotio_0p60_sc4, 'bo', label = 'ubdt > 0.60')
ax1.plot(edges_0p65_sc4, rotio_0p65_sc4, 'ro', label = 'ubdt > 0.65')
ax1.plot(edges_0p70_sc4, rotio_0p70_sc4, 'go', label = 'ubdt > 0.70')
ax1.set_ylim(-5,5)
ax1.grid()
ax1.legend(fontsize=18)
ax1.set_ylabel('Bkg eff', fontsize=18)
ax1.set_xlabel('$M_{T,J_2}$ (GeV)', fontsize=18)
plt.savefig('ubdt_vs_bdt_study/uBDTeff_vs_MT_scenario4.png')



In [ ]:
n_sc1 = X_test_ext_sc1.shape[1]-1
n_sc2 = X_test_ext.shape[1]-1
n_sc3 = X_test_ext_sc3.shape[1]-1
n_sc4 = X_test_ext_sc4.shape[1]-1
n_bdt = X_test_ext.shape[1]-1

bkg_ubdt_eff_sc1 = np.zeros(5)
bkg_ubdt_eff_sc2 = np.zeros(5)
bkg_ubdt_eff_sc3 = np.zeros(5)
bkg_ubdt_eff_sc4 = np.zeros(5)
bkg_bdt_eff = np.zeros(5)

sig_ubdt_eff_sc1 = np.zeros(5)
sig_ubdt_eff_sc2 = np.zeros(5)
sig_ubdt_eff_sc3 = np.zeros(5)
sig_ubdt_eff_sc4 = np.zeros(5)
sig_bdt_eff = np.zeros(5)

cuts = np.array([0.5, 0.55, 0.6, 0.65, 0.7])

for i in range(0,len(cuts)):
    bkg_ubdt_eff_sc1[i] = len(X_test_ext_sc1[:,n_sc1][(y_test_ext_sc1==0) & (y_prob_ext_sc1>cuts[i])])/len(X_test_ext_sc1[:,n_sc1][y_test_ext_sc1==0])
    bkg_ubdt_eff_sc2[i] = len(X_test_ext[:,n_sc2][(y_test_ext==0) & (y_prob_ext>cuts[i])])/len(X_test_ext[:,n_sc2][y_test_ext==0])
    bkg_ubdt_eff_sc3[i] = len(X_test_ext_sc3[:,n_sc3][(y_test_ext_sc3==0) & (y_prob_ext_sc3>cuts[i])])/len(X_test_ext_sc3[:,n_sc3][y_test_ext_sc3==0])
    bkg_ubdt_eff_sc4[i] = len(X_test_ext_sc4[:,n_sc4][(y_test_ext_sc4==0) & (y_prob_ext_sc4>cuts[i])])/len(X_test_ext_sc4[:,n_sc4][y_test_ext_sc4==0])
    bkg_bdt_eff[i]     = len(X_test_ext[:,n_bdt][(y_test==0) & (y_prob>cuts[i])])/len(X_test_ext[:,n_bdt][y_test==0])

    sig_ubdt_eff_sc1[i] = len(X_test_ext_sc1[:,n_sc1][(y_test_ext_sc1==1) & (y_prob_ext_sc1>cuts[i])])/len(X_test_ext_sc1[:,n_sc1][y_test_ext_sc1==1])
    sig_ubdt_eff_sc2[i] = len(X_test_ext[:,n_sc2][(y_test_ext==1) & (y_prob_ext>cuts[i])])/len(X_test_ext[:,n_sc2][y_test_ext==1])
    sig_ubdt_eff_sc3[i] = len(X_test_ext_sc3[:,n_sc3][(y_test_ext_sc3==1) & (y_prob_ext_sc3>cuts[i])])/len(X_test_ext_sc3[:,n_sc3][y_test_ext_sc3==1])
    sig_ubdt_eff_sc4[i] = len(X_test_ext_sc4[:,n_sc4][(y_test_ext_sc4==1) & (y_prob_ext_sc4>cuts[i])])/len(X_test_ext_sc4[:,n_sc4][y_test_ext_sc4==1])
    sig_bdt_eff[i]     = len(X_test_ext[:,n_bdt][(y_test==1) & (y_prob>cuts[i])])/len(X_test_ext[:,n_bdt][y_test==1])


    
print(bkg_ubdt_eff_sc4)
print(sig_ubdt_eff_sc4)

In [ ]:
eff_bkg_ext_sc4, eff_sig_ext_sc4, cuts_ext_sc4 = roc_curve(y_test_ext_sc4, y_prob_ext_sc4)


fig = plt.figure(figsize=(10,6))
ax = fig.gca()
ax.plot([0,1], [0,1], c='gray')
ax.plot(eff_bkg_ext_sc4, eff_sig_ext_sc4, c='green', label='uBDT Scenario IV = {:.5f}'.format(auc(eff_bkg_ext_sc4, eff_sig_ext_sc4)))
ax.plot(bkg_ubdt_eff_sc4,sig_ubdt_eff_sc4,'ro')
for index in range(len(bkg_ubdt_eff_sc4)):
    ax.text(bkg_ubdt_eff_sc4[index], sig_ubdt_eff_sc4[index], cuts[index], size=22)

#ax.set_ylim(0., 1.)
#ax.set_xlim(0., 1.)
ax.set_xlabel('Bkg efficiency', fontsize=18)
ax.set_ylabel('Sig efficiency', fontsize=18)
ax.legend(fontsize=18)
ax.grid()
ax.set_title('ROC: BDT & uBDT', fontsize=18)
plt.savefig('ubdt_vs_bdt_study/roc_ubdt_sc4.png')

